In [ ]:
#default_exp perception

## Google Colab preparations

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_MAIN = __name__ == '__main__'

In [ ]:
#Infrastructure for copying notebooks
if IN_COLAB and IN_MAIN:
    home_dir = '/content/drive/MyDrive/Colab Notebooks/Ecosystems/v3'
if IN_COLAB and IN_MAIN:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append(home_dir)
    !pip -q install import-ipynb
    %cd $home_dir

In [ ]:
#export
# import import_ipynb
from ecotwins.utility import normalize, distance

In [ ]:
#export
import numpy as np
from numpy.random import default_rng
from dataclasses import dataclass
from dataclasses import field

RADIUS = 4
ORIGIN = np.array([0.,0.])

In [ ]:
#export
@dataclass
class Perception:

    radius: float = RADIUS
    origin: np.ndarray = field(default_factory=lambda: ORIGIN.copy())
    steven_exponent: float = 0.6
    reward_radius: float = 0.0
    epsilon: float = np.sqrt(1e-1)
    use_clip: bool = False
    # Normally distributed noise in multi_direction and single_direction expressed
    # as standard deviation.
    direction_noise: float = 0 

    # def __init__(self, radius=RADIUS, origin=ORIGIN, steven_exponent=0.6, reward_radius=0):
    #     self.radius = radius
    #     self.origin = origin.copy()
    #     self.steven_exponent = steven_exponent
    #     self.reward_radius = reward_radius

    def __post_init__(self):
       self._rng = default_rng() 
        
    # Inverse square law

    # The stimulus caused by one object at distance d.
    # The closer the object the smaller the distance and the bigger the stimulus
    # There is a max stimulus (max firing) that one object can cause. We set it to 1.

    def stimulus(self, d, radius=None):
        if radius is None:
            radius = self.radius
        # epsilon = 1e-1
        # epsilon = np.sqrt(1e-1)
        d = np.maximum(d - self.reward_radius, 0)
        v = 1 / (self.epsilon + d) ** 2
        v[d > radius] = 0
        if self.use_clip:
            v[v > 1] = 1
        return v

    # This function sums the stimuli from all the objects.
    # def stimuli(distances, radius=RADIUS):
    def stimuli(self, distances):
        return self.stimulus(distances, self.radius).sum(axis=0)

    # Weber-Fechner's law
    def weber(self, stimuli):
        return np.log(1 + stimuli)

    # Steven's law
    # def steven(stimuli,exponent=0.6): # The exponent for smell in humans is 0.6
    def steven(self, stimuli):
        return stimuli ** self.steven_exponent

    # The sensation of the aggregated stimuli from objects inside the radius.
    def sensation(self, objects, position=None, radius=None):
        if position is None:
            position = self.origin
        if radius is None:
            radius = self.radius
        total = self.stimuli(distance(position, objects))
        return self.weber(total)
    
    def _noise_rotation(self):
        phi = self._rng.normal(scale=self.direction_noise) * 2 * np.pi
        c, s = np.cos(phi), np.sin(phi)
        return np.array([[c, -s], [s, c]])

    # Computes the main direction to the objects within the radius, i.e. the sum of the perception vectors.
    # Possible input to the policy network
    def multi_direction(self, position, objects):
        signals = self.stimulus(distance(position, objects)).reshape(-1, 1)
        vector_sum = (signals * normalize(objects - position)).sum(
            axis=0, keepdims=True
        )  # ta inte bort dim 1x2
        vector_sum = normalize(vector_sum).reshape(-1)
        return np.dot(self._noise_rotation(), vector_sum)
#         return normalize(vector_sum).reshape(-1)  # platta ut!

    # Total smell from all objects
    def total_intensity(self, position, objects):
        if objects.size == 0:
            return np.array([0.0])
        return self.sensation(objects,position)

    # Total smell from the closest object (disregarding all others)
    def nearest_intensity(self, position, objects):
        closest_point = self.closest(position, objects)
        return self.total_intensity(position, closest_point)

    # Filters out the visible objects from position.
    def visible(self, position, objects, radius=None):
        if radius is None:
            radius = self.radius
        return np.array([x for x in objects if distance(position, x) < radius])
        # return objects[distance(position, objects) < radius] #Faster!

    # This function returns the closest point to position in objects or the empty array
    def closest(self, position, objects, radius=None):
        if radius is None:
            radius = self.radius
        vis = self.visible(position, objects, radius)
        if vis.shape[0] == 0:
            return np.array([])  # Return an empty array if no objects are in sight
        index = np.argmin(distance(position, vis))
        return vis[index].reshape(1, 2)

    # Computes the direction to the closest point from position among objects within the radius
    # Returns the 0-vector if there are no such objects
    def closest_direction(self, position, objects):
        c = self.closest(position, objects)
        if c.shape == (0,):
            return np.zeros(2)
        else:
            v = normalize(c - position).reshape(-1)
            return np.dot(self._noise_rotation(), v)
#             return normalize(c - position).reshape(-1)

In [ ]:
p = Perception()
p.visible(np.array([.9,.9]), np.random.random((10,2)), radius=0)

array([], dtype=float64)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted animal_classes.ipynb.
Converted animation_helper.ipynb.
Converted ecoenv.ipynb.
Converted happiness.ipynb.
Converted index.ipynb.
Converted perception.ipynb.
Converted reflex_agent.ipynb.
Converted tyckande.ipynb.
Converted utility.ipynb.
Converted worlds.ipynb.
